## Step 2 : Multipole expansion

When we have a big number of particles, do direct sumations is computationally expensive. Instead of calculating all pair interactions, we will applied a tree code method. With this method we treat by direct sumation only the particles that are close to the target, and the ones that are in distant cells are treated as a single *pseudo-particle* centered at the cell's center of mass by applying a multipole expansion. 

In this notebook we will show the difference beteween doing multiple expansion and direct sumation for different target particles. 

First, we introduce multipole expansion at a cell's center. In a word, we want to decouple the interaction term $\frac{m_j}{r_{ij}}$ for the particles in a distance:

$$\begin{equation}
\Phi_i = \sum_{j=0}^N{\frac{m_j}{r_{ij}}} = \sum_{j=0}^N {A_j B_i}
\end{equation}$$

where $A_j$ is the term that only depends on source particles (*multipole*), and $B_i$ is the part depends on target particles (*weight*).

Recall the second-order taylor expansion in three variables $x$, $y$, $z$:

$$\begin{equation}\begin{split}
f(x,y,z) & \approx f(a,b,c) + \left(x-a\right)f_x(a,b,c) + \left(y-b\right)f_y(a,b,c) + \left(z-c\right)f_z(a,b,c) \\
& + \frac{1}{2!}[\left(x-a\right)^2f_{xx}(a,b,c) + \left(y-b\right)^2f_{yy}(a,b,c) + \left(z-c\right)^2f_{zz}(a,b,c) \\
& + 2\left(x-a\right)\left(y-b\right)f_{xy}(a,b,c) + 2\left(y-b\right)\left(z-c\right)f_{yz}(a,b,c) \\ 
& + 2\left(z-c\right)\left(x-a\right)f_{zx}(a,b,c)]
\end{split}\end{equation}$$

We assume that: 
$$f(x_j,y_j,z_j) = \frac{m_{j}}{r_{ij}} = \frac{m_{j}}{\sqrt{(x_i-x_j)^2 + (y_i-y_j)^2 + (z_i-z_j)^2}}$$ 

<img src="image/multipole.png">

Here we expand the interaction term $f(x_j,y_j,z_j)$ around a center point $(x_c, y_c, z_c)$ to the second order: 

$$\begin{equation}\begin{split}
f(x_j,y_j,z_j) & \approx f(x_c,y_c,z_c) + (x_j-x_c)\, f_{x_j}\rvert_c + (y_j-y_c)\, f_{y_j}\rvert_c + (z_j-z_c)\, f_{z_j}\rvert_c \\
& + \frac{1}{2!}[(x_j-x_c)^2f_{x_jx_j}\rvert_c + (y_j-y_c)^2f_{y_jy_j}\rvert_c + (z_j-z_c)^2f_{z_jz_j}\rvert_c \\
& + 2(x_j-x_c)(y_j-y_c)f_{x_jy_j}\rvert_c + 2(y_j-y_c)(z_j-z_c)f_{y_jz_j}\rvert_c \\ 
& + 2(z_j-z_c)(x_j-x_c)f_{z_jx_j}\rvert_c]
\end{split}\end{equation}$$

then we calculate each derivative's value at the expansion center $(x_c,y_c,z_c)$:

$$\begin{equation}\begin{split}
& f_{x_j}\rvert_c = \frac{x_i-x_c}{R^3} m_j\\
& f_{x_jx_j}\rvert_c = \left(\frac{3(x_i-x_c)^2}{R^5} - \frac{1}{R^3}\right) m_j \\
& f_{x_jy_j}\rvert_c = \frac{3(x_i-x_c)(y_i-y_c)}{R^5} m_j
\end{split}\end{equation}$$

where $R$ is the distance between target particle $i$ and the cell's center $c$. Now let's write out the interaction term again:

$$\begin{equation}\begin{split}
\frac{m_j}{r_{ij}} & \approx \frac{m_j}{R} + m_j(x_c-x_j)\left(-\frac{x_i-x_c}{R^3}\right) + m_j(y_c-y_j)\left(-\frac{y_i-y_c}{R^3}\right) \\
& + m_j(z_c-z_j)\left(-\frac{z_i-z_c}{R^3}\right) + \frac{m_j(x_c-x_j)^2}{2}\left(\frac{3(x_i-x_c)^2}{R^5} - \frac{1}{R^3}\right) \\
& + \frac{m_j(y_c-y_j)^2}{2}\left(\frac{3(y_i-y_c)^2}{R^5} - \frac{1}{R^3}\right) + \frac{m_j(z_c-z_j)^2}{2}\left(\frac{3(z_i-z_c)^2}{R^5} - \frac{1}{R^3}\right) \\
& + \frac{m_j(x_c-x_j)(y_c-y_j)}{2}\frac{3(x_i-x_c)(y_i-y_c)}{R^5} \\
& + \frac{m_j(y_c-y_j)(z_c-z_j)}{2}\frac{3(y_i-y_c)(z_i-z_c)}{R^5} \\
& + \frac{m_j(z_c-z_j)(x_c-x_j)}{2}\frac{3(z_i-z_c)(x_i-x_c)}{R^5}
\end{split}\end{equation}$$

The multipoles have no relationship with target particles, so we can calculate those constants before looping in $i$. The second part in each term only depends on target particles $i$, so we can calculate this part in the loop. From above we can write the interaction term as:

$$\begin{equation}
\frac{m_j}{r_{ij}} = \sum_{k=1}^{10} A_k(j)\,B_k(i)
\end{equation}$$

Remember that what we just considered is only one source particle $j$'s effect on target $i$. In order to evaluate the potential $\Phi_i$, we need to take all the source particles into account:

$$\begin{equation}\begin{split}
\Phi_i & = \sum_{j=0}^N{\frac{m_j}{r_{ij}}} = \sum_{j=0}^N \sum_{k=1}^{10} A_k(j)\,B_k(i) \\
& = \sum_{k=1}^{10} \left(\sum_{j=0}^N A_k(j)\right) B_k(i)
\end{split}\end{equation}$$

In [1]:
# importing libraries
import numpy
from treecode_helper import Particle, Point
from matplotlib import pyplot, rcParams
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

# customizing plot parameters
rcParams['figure.dpi'] = 100
rcParams['font.size'] = 14
rcParams['font.family'] = 'StixGeneral'

In [4]:
n = 50     # number of particles
m = 1./n    # mass

# initialize sources and targets
source_coords = numpy.random.random((n, 3)).tolist()
target_coords = (-numpy.random.random((n, 3))).tolist()
sources = [ Particle(coord, m=m) for coord in source_coords ]
targets = [ Particle(coord, m=m) for coord in target_coords ]

In [3]:
center = Point([0.5, 0.5, 0.5])